# EHT Tutorial 2: Imaging M87* from Real Data
This tutorial is an abridged version of the original pipeline (see `eht-imaging/eht-imaging_pipeline.py` at https://github.com/eventhorizontelescope/2019-D01-02). We skip reverse tapering, pre-calibration, and the last two rounds of imaging, but the result is already close to the result of the original pipeline.

## Environment setup

In [ ]:
# Install `eht-imaging` library.
!pip install ehtim

In [ ]:
# Download the publicly-available UVFITS data sets.
!wget https://de.cyverse.org/anon-files/iplant/home/shared/commons_repo/curated/EHTC_FirstM87Results_Apr2019/uvfits/SR1_M87_2017_095_hi_hops_netcal_StokesI.uvfits
!wget https://de.cyverse.org/anon-files/iplant/home/shared/commons_repo/curated/EHTC_FirstM87Results_Apr2019/uvfits/SR1_M87_2017_095_lo_hops_netcal_StokesI.uvfits
!wget https://de.cyverse.org/anon-files/iplant/home/shared/commons_repo/curated/EHTC_FirstM87Results_Apr2019/uvfits/SR1_M87_2017_096_hi_hops_netcal_StokesI.uvfits
!wget https://de.cyverse.org/anon-files/iplant/home/shared/commons_repo/curated/EHTC_FirstM87Results_Apr2019/uvfits/SR1_M87_2017_096_lo_hops_netcal_StokesI.uvfits
!wget https://de.cyverse.org/anon-files/iplant/home/shared/commons_repo/curated/EHTC_FirstM87Results_Apr2019/uvfits/SR1_M87_2017_100_hi_hops_netcal_StokesI.uvfits
!wget https://de.cyverse.org/anon-files/iplant/home/shared/commons_repo/curated/EHTC_FirstM87Results_Apr2019/uvfits/SR1_M87_2017_100_lo_hops_netcal_StokesI.uvfits
!wget https://de.cyverse.org/anon-files/iplant/home/shared/commons_repo/curated/EHTC_FirstM87Results_Apr2019/uvfits/SR1_M87_2017_101_hi_hops_netcal_StokesI.uvfits
!wget https://de.cyverse.org/anon-files/iplant/home/shared/commons_repo/curated/EHTC_FirstM87Results_Apr2019/uvfits/SR1_M87_2017_101_lo_hops_netcal_StokesI.uvfits

In [ ]:
# Import libraries.
import ehtim as eh
import numpy as np

## Fixed parameters

In [ ]:
# Fiducial imaging parameters from the eht-imaging parameter survey:
zbl = 0.6                        # total compact flux density in Jy
gauss_fwhm = 40. * eh.RADPERUAS  # FWHM of Gaussian prior in radians

In [ ]:
# Fixed imaging parameters:
ttype = 'fast'             # type of Fourier transform (direct | nfft | fast)
                           # NOTE: the original pipeline uses 'nfft',
                           # but we don't have PyNFFT installed.
npix = 64                  # number of pixels across the reconstructed image
fov = 128 * eh.RADPERUAS   # FOV of the reconstructed image in radians
maxit = 100                # maximum number of convergence iterations
                           # for imager
stop = 1e-4                # imager stopping criterion
uv_zblcut = 0.1e9          # uv-distance that separates the inter-site
                           # "zero"-baselines from intra-site baselines

In [ ]:
# Constant regularization weights:
reg_term = {
  'simple': 100,  # maximum entropy
  'tv': 1.,       # total variation
  'tv2': 1.,      # total squared variation
  'flux': 1e4     # compact flux constraint
}

# Inflate amplitude error bars with systematic noise, which accounts for
# gain errors, polarization leakage, etc.
systematic_noise = {
    'AA': 0.01220,
    'AP': 0.01339,
    'AZ': 0.00860,
    'LM': 0.17613,  # LMT had the most variability
    'PV': 0.01220,
    'SM': 0.01810,
    'JC': 0.01693,
    'SP': 0.00860
}

## Data preparation

### Load and prepare the data

In [ ]:
# Load both high- and low-band datasets.
day = '095'  # day of observation ('095', '096', '100', or '101')
obspath1 = f'SR1_M87_2017_{day}_hi_hops_netcal_StokesI.uvfits'
obspath2 = f'SR1_M87_2017_{day}_lo_hops_netcal_StokesI.uvfits'

obs1 = eh.obsdata.load_uvfits(obspath1)
obs2 = eh.obsdata.load_uvfits(obspath2)

In [ ]:
# Average the data based on individual scan lengths.
obs1.add_scans()  # add scans
obs2.add_scans()
obs1 = obs1.avg_coherent(0., scan_avg=True)  # scan-average
obs2 = obs2.avg_coherent(0., scan_avg=True)

# Add a slight offset to avoid forming closure quantities between
# the two datasets.
obs2.data['time'] += 0.00001

# Concatenate the observations into a single observation object.
obs = obs1.copy()
obs.data = np.concatenate([obs1.data, obs2.data])

In [ ]:
# Estimate the total flux density from the ALMA(AA) -- APEX(AP) zero baseline
zbl_tot = np.median(obs.unpack_bl('AA', 'AP', 'amp')['amp'])

# Rescale short baselines to excise contributions from extended flux.
def rescale_zerobaseline(obs, totflux, orig_totflux, uv_max):
  multiplier = totflux / orig_totflux
  for j in range(len(obs.data)):
    if (obs.data['u'][j]**2 + obs.data['v'][j]**2)**0.5 >= uv_max:
      continue
    for field in ['vis', 'qvis', 'uvis', 'vvis', 'sigma', 'qsigma',
                  'usigma', 'vsigma']:
      obs.data[field][j] *= multiplier

if zbl != zbl_tot:
  rescale_zerobaseline(obs, zbl, zbl_tot, uv_zblcut)

In [ ]:
# Flag out sites in the obs.tarr table with no measurements.
allsites = set(obs.unpack(['t1'])['t1']) | set(obs.unpack(['t2'])['t2'])
obs.tarr = obs.tarr[[o in allsites for o in obs.tarr['site']]]
obs = eh.obsdata.Obsdata(
  obs.ra, obs.dec, obs.rf, obs.bw, obs.data, obs.tarr,
  source=obs.source, mjd=obs.mjd)

# Order the stations by SNR.
# This will create a minimal set of closure quantities
# with the highest SNR and smallest covariance.
obs.reorder_tarr_snr()

In [ ]:
# Save preprocessed Obsdata.
obs.save_uvfits(f'obs_{day}_preprocessed.uvfits')

## Image reconstruction

In [ ]:
# Make an image of a Gaussian blob for initialization and
# maximum entropy regularization.
gaussim = eh.image.make_square(obs, npix, fov)
gaussim = gaussim.add_gauss(zbl, (gauss_fwhm, gauss_fwhm, 0, 0, 0))

# To avoid gradient singularities in the first step, add an additional small
# Gaussian.
gaussim = gaussim.add_gauss(
  zbl * 1e-3, (gauss_fwhm, gauss_fwhm, 0, gauss_fwhm, gauss_fwhm))

# Get intrinsic resolution for blurring images between imaging rounds.
res = obs.res()

In [ ]:
# Define a helper function for running coarse-to-fine imaging.
def converge(imgr, major=3, blur_frac=1.0):
  """Repeat imaging with blurring to assure good convergence."""
  for _ in range(major):
    init = imgr.out_last().blur_circ(blur_frac * res)
    imgr.init_next = init
    imgr.make_image_I(show_updates=False)

### Imaging round 1

In [ ]:
data_term = {'amp': 0.2, 'cphase': 1., 'logcamp': 1.}

imgr = eh.imager.Imager(
  obs,
  init_im=gaussim,
  prior_im=gaussim,
  flux=zbl,
  data_term=data_term,
  maxit=maxit,
  norm_reg=True,
  systematic_noise=systematic_noise,
  reg_term=reg_term,
  ttype=ttype,
  cp_uv_min=uv_zblcut,
  stop=stop)

imgr.make_image_I(show_updates=False)
converge(imgr)

In [ ]:
# Look at the result after the first round of imaging (before self-calibration).
out = imgr.out_last()
out.display();

In [ ]:
# Self-calibrate to the result of the first round of imaging (phase-only).
# The solution_interval is 0 to align phases from high and low bands if needed.
obs_sc = eh.selfcal(
    obs, imgr.out_last(), method='phase', ttype=ttype, solution_interval=0.0)

In [ ]:
# Compare phases before vs. after self-calibration.
eh.plotting.comp_plots.plotall_obs_compare(
  [obs, obs_sc], 'uvdist', 'phase', ebar=False,
  legendlabels=['original', 'self-calibrated']);

### Imaging round 2

In [ ]:
# Blur the previous reconstruction to the intrinsic resolution.
init = imgr.out_last().blur_circ(res)

# Increase the weights on the data terms.
data_term_intermediate = {
  'vis': imgr.dat_terms_last()['amp'] * 10,
  'cphase': imgr.dat_terms_last()['cphase'] * 10,
  'logcamp': imgr.dat_terms_last()['logcamp'] * 10
}

In [ ]:
# Reinitialize the imager.
imgr = eh.imager.Imager(
  obs_sc,
  init_im=init,
  prior_im=gaussim,
  flux=zbl,
  data_term=data_term_intermediate,
  maxit=maxit,
  norm_reg=True,
  systematic_noise=systematic_noise,
  reg_term=reg_term,
  ttype=ttype,
  cp_uv_min=uv_zblcut,
  stop=stop)

imgr.make_image_I(show_updates=False)
converge(imgr)

In [ ]:
# Look at the result after the second round of imaging.
out = imgr.out_last()
out.display();